# Check model
1. Load the postprocessing dataset
2. Modify each label is for each sample, not each timestep
3. Run the model

In [1]:
import tensorflow as tf
import numpy as np
import collections
import os
import argparse
import datetime as dt
import random
import scipy.io as spio
import pandas as pd
import matplotlib.pyplot as plt
import datetime
 
import keras
from keras import optimizers
import keras.backend as K
from keras.models import Sequential, Model, load_model
from keras.layers import GRU, LSTM, Dense, TimeDistributed, Flatten, Input, Dropout, Reshape, Concatenate
from keras.layers.convolutional import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D
from sklearn.utils import shuffle, class_weight
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, fbeta_score
from sklearn.model_selection import train_test_split

C:\nxt\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load dataset

In [2]:
class xt_dataset:
    def __init__(self, n_timestep, n_feature, dataset_id):
        self.n_timestep = n_timestep
        self.n_feature = n_feature
        self.id = dataset_id
    
    # ==========================================================
    # Load postprocessing dataset from file
    # and store them in arrays X_train, y_train, X_test, y_test
    # ==========================================================
    def load(self, dir_in, fileID, stateful_rnn, batch_size):
        print('---- INFO: Load dataset:')        
        self.X_train = []
        self.y_train = []
        self.X_val = []
        self.X_test = []
        self.y_test = []
        tmp_y_train = []
        tmp_y_test = []
        
        file_train = dir_in + 'train_' + str(fileID)
        file_test = dir_in + 'test_' + str(fileID)

        self.X_train = np.loadtxt(file_train, delimiter=' ', usecols=(range(0, self.n_feature)))
        tmp_y_train = np.loadtxt(file_train, delimiter=' ', usecols=(self.n_feature))
        self.X_test = np.loadtxt(file_test, delimiter=' ', usecols=(range(0, self.n_feature)))
        tmp_y_test = np.loadtxt(file_test, delimiter=' ', usecols=(self.n_feature))
        
        
        # ------------------
        d1, d2 = self.X_train.shape
        n_sample = int(d1/self.n_timestep)
        n_new_sample = n_sample
        #print(d1, d2, self.n_timestep, n_sample)
        self.X_train = self.X_train.reshape((n_sample, self.n_timestep, self.n_feature))
        
        if stateful_rnn == True:
            # Fit to 2^n batches
            n_new_sample = int(n_sample / batch_size) * batch_size
            self.X_train = self.X_train[0:n_new_sample]
        
        # Modify label from each timestep to sample
        for i in range (0, d1, self.n_timestep):
            self.y_train.append(tmp_y_train[i])
        self.y_train = np.asarray(self.y_train)        
        self.y_train = self.y_train.reshape((n_sample, 1))
        
        if stateful_rnn == True:
            # Fit to 2^n batches
            self.y_train = self.y_train[0:n_new_sample]
        
        print('------ Old, new number of TRAIN samples =', n_sample, n_new_sample)

        # ------------------
        d1, d2 = self.X_test.shape
        n_sample = int(d1/self.n_timestep)
        n_new_sample = n_sample
        self.X_test = self.X_test.reshape((n_sample, self.n_timestep, self.n_feature))
        
        if stateful_rnn == True:
            # Fit to 2^n batches
            n_new_sample = int(n_sample / batch_size) * batch_size
            self.X_test = self.X_test[0:n_new_sample]
        
        # Modify label from each timestep to sample
        for i in range (0, d1, self.n_timestep):
            self.y_test.append(tmp_y_test[i])
        self.y_test = np.asarray(self.y_test)        
        self.y_test = self.y_test.reshape((n_sample, 1))
        
        if stateful_rnn == True:
            # Fit to 2^n batches
            self.y_test = self.y_test[0:n_new_sample]
        
        print('------ Old, new number of TEST samples =', n_sample, n_new_sample)

        print('------ Shapes of X_train, y_train, X_test, y_test =', \
              self.X_train.shape, self.y_train.shape, self.X_test.shape, self.y_test.shape)     
        
# ===================================================
# Test function
dog = xt_dataset(400, 16, 'Dog_1')  # timesteps, features
dog.load('D:\\Romanlab\XT_DataSet\dataset2\Dog_1OC\\', 0, True, 64)

---- INFO: Load dataset:
------ Old, new number of TRAIN samples = 758 704
------ Old, new number of TEST samples = 120 64
------ Shapes of X_train, y_train, X_test, y_test = (704, 400, 16) (704, 1) (64, 400, 16) (64, 1)


In [3]:
import keras
from sklearn.metrics import roc_auc_score

# https://github.com/keunwoochoi/keras_callbacks_example
class xt_best_model(keras.callbacks.Callback):
    
    def config(self, batch_size, best_model_name, X, y):
        self.max_auc = -1
        self.max_sens = -1
        self.max_f1score = -1
        self.max_f2score = -1
        self.min_loss = 1
        self.prev_loss = 1
        
        self.test_train_max_auc = -1 
        self.test_max_auc = -1
        self.test_max_sens = -1
        self.test_max_f1score = -1
        self.test_max_f2score = -1
        
        self.X = X
        self.y = y
        self.best_model_name = best_model_name
        self.bs = batch_size
        
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.losses = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        
        # ----- Train -----
        if self.bs > 0:
            y_pred = self.model.predict(self.X, batch_size=self.bs)
        else:
            y_pred = self.model.predict(self.X)
        
        dim = self.y.shape 
        y_pred_ = y_pred.reshape(dim[0] * dim[1])
        y_pred_ = np.round(y_pred_)
        y_ = self.y.reshape(dim[0] * dim[1])        

        current_auc = roc_auc_score(y_, y_pred_)        
        current_tn, current_fp, current_fn, current_tp = confusion_matrix(y_, y_pred_).ravel()
        current_sens = current_tp/(current_tp + current_fn)
        
        current_f1score = f1_score(y_, y_pred_)  
        current_f2score = fbeta_score(y_, y_pred_, beta=2)  
        
        current_loss = logs.get('loss')
        
        # Check F2 Score first
        #if self.max_f2score < current_f2score:
        #    print('Update new model: F2 Score =', self.max_f2score, '-->', current_f2score)
        #    self.max_f2score = current_f2score
        #    self.max_f1score = current_f1score
        #    self.max_auc = current_auc
        #    self.max_sens = current_sens
        #    self.model.save(self.best_model_name)  
        
        # Check F1 Score first
        #if self.max_f1score < current_f1score:
        #    print('Update new model: F1 Score =', self.max_f1score, '-->', current_f1score)
        #    self.max_f1score = current_f1score
        #    self.max_auc = current_auc
        #    self.max_sens = current_sens
        #    self.model.save(self.best_model_name)
        
        # Check AUC
        #elif self.max_f2score == current_f2score:  
        flag = 0
        #if (self.max_auc < current_auc) or (self.max_auc == current_auc and self.min_loss > current_loss):
        if (current_loss < self.prev_loss) or (current_loss >= self.prev_loss and self.max_auc < current_auc):
        #if (current_loss < self.min_loss) or ((current_loss / self.min_loss < 2) and current_auc >= self.max_auc):
        #if (current_loss < self.min_loss or (current_loss >= self.min_loss and current_loss/self.min_loss < 1.1)) \
        #        and current_auc >= self.max_auc:
            print('Update new train model: AUC =', self.max_auc, '-->', current_auc)
            self.max_f2score = current_f2score
            self.max_f1score = current_f1score
            self.max_auc = current_auc
            self.max_sens = current_sens
            self.min_loss = current_loss
            previous_loss = current_loss
            self.model.save(self.best_model_name)  
            flag = 1
            
        self.prev_loss = current_loss
        
        print('TRAIN: Current SENS =', current_sens, ' Max SENS =', self.max_sens, \
               ' Current AUC =', current_auc, ' Max AUC =', self.max_auc, \
                ' Current F1 =', current_f1score, ' Max F1 =', self.max_f1score, \
                 ' Current F2 =', current_f2score, ' Max F2 =', self.max_f2score, \
                  ' Current Loss =', current_loss, ' Min Loss =', self.min_loss \
             )
        

        # Then check SENS
        #elif self.max_auc == current_auc:
        #    if self.max_sens < current_sens:
        #        print('Update new model: SENS =', self.max_sens, '-->', current_sens)
        #        self.max_f2score = current_f2score
        #        self.max_f1score = current_f1score
        #        self.max_auc = current_auc
        #        self.max_sens = current_sens
        #        self.model.save(self.best_model_name)
        
        #self.aucs.append(current_auc)
        
        # ---- Test ----
        X = self.validation_data[0]
        y = self.validation_data[1]
        
        if self.bs > 0:
            y_pred = self.model.predict(X, batch_size=self.bs)
        else:
            y_pred = self.model.predict(X)
        
        dim = y.shape 
        y_pred_ = y_pred.reshape(dim[0] * dim[1])
        y_pred_ = np.round(y_pred_)
        y_ = y.reshape(dim[0] * dim[1])        

        current_auc = roc_auc_score(y_, y_pred_)        
        current_tn, current_fp, current_fn, current_tp = confusion_matrix(y_, y_pred_).ravel()
        current_sens = current_tp/(current_tp + current_fn)
        
        current_f1score = f1_score(y_, y_pred_)  
        current_f2score = fbeta_score(y_, y_pred_, beta=2) 
        
        if self.test_max_auc < current_auc:
            print('Update new test AUC: AUC =', self.test_max_auc, '-->', current_auc)
            self.test_max_f2score = current_f2score
            self.test_max_f1score = current_f1score
            self.test_max_auc = current_auc
            self.test_max_sens = current_sens
        if flag == 1:
            self.test_train_max_auc = current_auc
            flag = 0
            
        print('TEST: Current SENS =', current_sens, ' Max SENS =', self.test_max_sens, \
               ' Current AUC =', current_auc, ' Max AUC: ', self.test_max_auc, \
                ' Current F1 =', current_f1score, ' Max F1: ', self.test_max_f1score, \
                 ' Current F2 =', current_f2score, ' Max F2: ', self.test_max_f2score, \
                  ' AUC at train max =', self.test_train_max_auc)     
            
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

# Model 7
- Two/Three hidden layers
- Hyperparameters: batch_size, number of hidden units/layer 1, 2, 3
- Class weight enable: 0/1
- CNN algorithm as front-end
- RNN algorithms: LSTM, GRU

In [6]:
class xt_model:
    # ==========================================
    def config(self, kfold, batch_size, rnn_algo, class_weight, n_hlayer, n_hunit, rnn_dropout):
        print('---- INFO: Set model:')       
        self.kfold = kfold
        self.bs = batch_size
        self.rnn_algo = rnn_algo   # LSTM, GRU
        self.cw = class_weight   # 1: turn on, 0: turn off
        self.n_hlayer = n_hlayer
        
        self.n_hunit = []
        self.rnn_dropout = []
        for i in range (0, self.n_hlayer):
            self.n_hunit.append(n_hunit[i])
            self.rnn_dropout.append(rnn_dropout[i])
            
        print('------ ' \
              'K-fold =', self.kfold, ', Batch size =', self.bs, \
              ', RNN algorithm =', self.rnn_algo, ', Class weight enable =', \
              self.cw, ', # hidden layers =', self.n_hunit, ', RNN dropout =', self.rnn_dropout),
               
    # ==========================================
    # Model v0_7a
    def run(self, dataset, k, stateful_rnn):
        print('---- INFO: Running: id.k.bs.cw.hu =', dataset.id, k, self.bs, self.cw, self.n_hunit),

        # ----------------------------------------------------
        # Create model
        # ----------------------------------------------------
        if stateful_rnn == True:
            input_layer = Input(batch_shape=(self.bs, dataset.n_timestep, dataset.n_feature ))
        else:
            input_layer = Input(shape=(dataset.n_timestep, dataset.n_feature ))

        #
        conv1 = Conv1D(filters=64,
           kernel_size=2,
           strides=1,
           activation='relu',
           padding='same')(input_layer)
        maxpool1 = MaxPooling1D(pool_size=8, 
              strides=4,
              padding='same')(conv1)
        
        #
        conv2 = Conv1D(filters=64,
           kernel_size=8,
           strides=1,
           activation='relu',
           padding='same')(input_layer)
        maxpool2 = MaxPooling1D(pool_size=8, 
              strides=4,
              padding='same')(conv2)
        
        #
        merge1 = Concatenate()([maxpool1, maxpool2])

        #
        conv11 = Conv1D(filters=128,
           kernel_size=4,
           strides=1,
           activation='relu')(merge1)
        maxpool11 = MaxPooling1D(pool_size=4, 
           strides=2)(conv11)

        #
        lstm1 = LSTM(32)(maxpool11)
        dense1 = Dense(1, activation='sigmoid')(lstm1)
        
        model = Model(inputs=input_layer, outputs=dense1)
        model.summary()
        
        # ==========================================
        # define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
        # Ref: https://stackoverflow.com/questions/41032551/
        # how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras
        #def recall(y_true, y_pred):
        #    # reset the local variables
        #    #tf.local_variables_initializer()
        #    # any tensorflow metric
        #    dim = tf.shape(y_true)
        #    y_true = tf.reshape(y_true, [(dim[0] * dim[1]), dim[2]])
        #    y_pred = tf.round(tf.reshape(y_pred, [(dim[0] * dim[1]), dim[2]]))
        #    #value, update_op = tf.metrics.auc(predictions=y_pred, labels=y_true)
        #    value, update_op = tf.metrics.recall(predictions=y_pred, labels=y_true)
            
        #    # find all variables created for this metric
        #    metric_vars = [i for i in tf.local_variables() if 'recall' in i.name.split('/')[1]]

        #    # Add metric variables to GLOBAL_VARIABLES collection.
        #    # They will be initialized for new session.
        #    for v in metric_vars:
        #        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

        #    # force to update metric values
        #    with tf.control_dependencies([update_op]):
        #        value = tf.identity(value)
        #        return value
    
        # ==========================================
        #adam = optimizers.Adam(lr=0.0001)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[recall])
        
        # Calculate class weight
        # Ref: https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-
        # for-imbalanced-classes-in-keras
        if self.cw == 1:
            dim = dataset.y_train.shape 
            y_train_ = dataset.y_train.reshape(dim[0] * dim[1])
        else:
            y_train_ = [0, 1]
        
        class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_), y_train_)
        #class_weights = [1, 10]
        print('------ Class_weights =', class_weights)
        
        # ----------------------------------------------------
        # Create call_back
        # ----------------------------------------------------
        now = datetime.datetime.now()
        sub_name = 'date--id.k.bs.cw.hu1.hu2.hu3--' \
            + (now.strftime("%Y.%m.%d.%H.%M--")) + dataset.id + '.' \
            + str(k) + '.' + str(self.bs) + '.' + str(self.cw) + '.' \
            + str(self.n_hunit[0]) + '.' + str(self.n_hunit[1]) + '.' + str(self.n_hunit[2])
        
        best_model_name = './Model_copy1/best-model--' + sub_name
        graph_name = './Graph_7a_copy1/' + sub_name

        # train LSTM
        # Ref: https://keras.io/callbacks/
        my_callback = xt_best_model()
        if stateful_rnn == True:
            my_callback.config(self.bs, best_model_name, dataset.X_train, dataset.y_train)
        else:
            my_callback.config(0, best_model_name, dataset.X_train, dataset.y_train)
        callbacks = [
            keras.callbacks.EarlyStopping(monitor='loss', #'val_loss', 
                                          patience=20, 
                                          min_delta=0.001,
                                          verbose=1,
                                          mode='min'),
            #keras.callbacks.ModelCheckpoint(filepath=best_model_name, 
            #                                monitor='val_recall', #'val_loss', 
            #                                save_best_only=True, 
            #                                verbose=1,
            #                                mode='max'),
            keras.callbacks.TensorBoard(log_dir=graph_name, 
                                        histogram_freq=0, 
                                        write_graph=True, 
                                        write_images=True,
                                        write_grads=False),
            my_callback
        ]

        # ----------------------------------------------------
        # Train model
        # ----------------------------------------------------
        # Ref: https://keras.io/models/model/
        history = model.fit(dataset.X_train, 
                              dataset.y_train, 
                              epochs=100, 
                              batch_size=self.bs, 
                              verbose=1, 
                              shuffle=not stateful_rnn,
                              callbacks=callbacks,
                              class_weight=class_weights,
                              validation_data=(dataset.X_test, dataset.y_test))
                              #validation_split=0.2)
        # plot history
        #pyplot.plot(history.history['loss'], label='train')
        #pyplot.plot(history.history['val_loss'], label='test')
        #pyplot.legend()
        #pyplot.show()

        # ----------------------------------------------------
        # Save results
        # ----------------------------------------------------
        #print('my_callback.losses=', my_callback.losses)
        #print('my_callback.aucs=', my_callback.aucs)

        self.train_loss, self.train_acc, self.test_loss, self.test_acc = [0, 0, 0, 0]
        self.tp, self.fn, self.tn, self.fp = [0, 0, 0, 0]
        self.sens, self.spec, self.auc, self.f1score, self.f2score = [0, 0, 0, 0, 0]
        
        # Load best model
        #best_model = load_model(best_model_name, custom_objects={'recall': recall})
        best_model = load_model(best_model_name)

        # Loss, Accuracy
        # Ref: https://keras.io/models/model/
        self.train_loss, self.train_acc = best_model.evaluate(dataset.X_train,
                                                              dataset.y_train,
                                                              batch_size=self.bs)
        self.test_loss, self.test_acc = best_model.evaluate(dataset.X_test, 
                                                            dataset.y_test,
                                                            batch_size=self.bs)

        # Ref: https://keras.io/models/model/
        y_pred = best_model.predict(dataset.X_test, 
                                    batch_size=self.bs, 
                                    verbose=0, 
                                    steps=None)

        # Flatten y_test and y_pred
        #print('------ Debug: y_test.shape =', y_test.shape)
        dim = dataset.y_test.shape 
        y_pred_ = y_pred.reshape(dim[0] * dim[1])
        y_pred_ = np.round(y_pred_)
        y_test_ = dataset.y_test.reshape(dim[0] * dim[1])

        # Save pred, test to file
        result_name = './Results_copy1/test-pred--' + sub_name
        np.savetxt(result_name, [y_test_, y_pred_], delimiter=' ', fmt = '%.1f')    

        # TN, FP, FN, TP, AUC, sensitivity, specificity, AUC
        # Ref: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
        self.tn, self.fp, self.fn, self.tp = confusion_matrix(y_test_, y_pred_).ravel()
        self.sens = self.tp/(self.tp + self.fn)
        self.spec = self.tn/(self.fp + self.tn)
        self.auc = roc_auc_score(y_test_, y_pred_)
        self.f1score = f1_score(y_test_, y_pred_)
        self.f2score = fbeta_score(y_test_, y_pred_, beta=2)

# Run

In [7]:
# ------------------------------------------------
# User setting
filter_en = 0
stateful_rnn = False # decide LSTM stateful or stateless
test_subject_id = 'Patient_8' # Dog_1, ..., Patient_1, ... , Dog_ (full), Patient_ (full)
dir_in = 'D:\\Romanlab\XT_DataSet\dataset2\\' + test_subject_id + 'OG\\'  

# ------------------------------------
# Initialize
if filter_en == 1:
    n_band = 5
else:
    n_band = 1
#
if test_subject_id == 'Patient_1':
    test_subject = xt_dataset(500, 68, test_subject_id) # timesteps, features
elif test_subject_id == 'Patient_2':
    test_subject = xt_dataset(500, 16, test_subject_id) # timesteps, features
elif test_subject_id == 'Patient_3':
    test_subject = xt_dataset(500, 55, test_subject_id) # timesteps, features
elif test_subject_id == 'Patient_4':
    test_subject = xt_dataset(500, 72, test_subject_id) # timesteps, features
elif test_subject_id == 'Patient_5':
    test_subject = xt_dataset(500, 64, test_subject_id) # timesteps, features
elif test_subject_id == 'Patient_6':
    test_subject = xt_dataset(500, 30, test_subject_id) # timesteps, features
elif test_subject_id == 'Patient_7':
    test_subject = xt_dataset(500, 36, test_subject_id) # timesteps, features  
elif test_subject_id == 'Patient_8':
    test_subject = xt_dataset(500, 16, test_subject_id) # timesteps, features  
else:
    test_subject = xt_dataset(400, 16*n_band, test_subject_id) # timesteps, features
model = xt_model()

# ------------------------------------ 
# Results are stored in result array that contains 13 dimensions:
# train_loss, train_acc, test_loss, test_acc, tp, fn, tn, fp, sensitivity, specificity, auc, f1-score, f2-score
results = []

kfold = 5
bs =  [ 64]
hu1 = [ 8, 16, 32, 64, 32, 36]
hu2 = [ 8, 16, 32, 64, 36, 8]
hu3 = [ 8, 16, 32]

for i in range (2, 3):    
    # kfold, batch_size, rnn_algo, class_weight, n_hlayer, n_hunit
    model.config(kfold, bs[0], 'LSTM', 1, 3, [hu1[i], 0, 0], [0, 0, 0])
    
    #for j in range (0, model.kfold):  
    for j in [0, 4, 1, 2, 3]:  
        for k in range (0, 3):  
        #for k in range (0, 1): 
            #
            test_subject.load(dir_in, j, stateful_rnn, bs[0])
            model.run(test_subject, j, stateful_rnn)

            #
            result = [model.train_loss, model.train_acc, model.test_loss, model.test_acc, \
                      model.tp, model.fn, model.tn, model.fp, \
                      model.sens, model.spec, model.auc, model.f1score, model.f2score]
            results.append(result)

            #
            print(result)
            np.savetxt('./Results_copy1/20180731_result07a.csv', results, delimiter=',')   # 

---- INFO: Set model:
------ K-fold = 5 , Batch size = 64 , RNN algorithm = LSTM , Class weight enable = 1 , # hidden layers = [32, 0, 0] , RNN dropout = [0, 0, 0]
---- INFO: Load dataset:
------ Old, new number of TRAIN samples = 2942 2942
------ Old, new number of TEST samples = 378 378
------ Shapes of X_train, y_train, X_test, y_test = (2942, 500, 16) (2942, 1) (378, 500, 16) (378, 1)
---- INFO: Running: id.k.bs.cw.hu = Patient_8 0 64 1 [32, 0, 0]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500, 16)      0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 500, 64)      2112        input_2[0][0]                    
_________________________________________________

Update new train model: AUC = 0.9787779486836532 --> 0.9818454304970389
TRAIN: Current SENS = 0.9644218551461246  Max SENS = 0.9644218551461246  Current AUC = 0.9818454304970389  Max AUC = 0.9818454304970389  Current F1 = 0.9815712900096993  Max F1 = 0.9815712900096993  Current F2 = 0.9712092130518234  Max F2 = 0.9712092130518234  Current Loss = 0.07576153514226791  Min Loss = 0.07576153514226791
TEST: Current SENS = 0.5277777777777778  Max SENS = 0.6666666666666666  Current AUC = 0.7638888888888888  Max AUC:  0.8260233918128654  Current F1 = 0.6909090909090909  Max F1:  0.7384615384615385  Current F2 = 0.5828220858895705  Max F2:  0.6936416184971099  AUC at train max = 0.7638888888888888
Epoch 8/100
2942/2942 [==============================] - 7s 2ms/step - loss: 0.0757 - acc: 0.9776 - val_loss: 0.3051 - val_acc: 0.9497
Update new train model: AUC = 0.9818454304970389 --> 0.980731755797796
TRAIN: Current SENS = 0.9695044472681067  Max SENS = 0.9695044472681067  Current AUC = 0.9807317

TEST: Current SENS = 0.5833333333333334  Max SENS = 0.6944444444444444  Current AUC = 0.7858187134502924  Max AUC:  0.8399122807017544  Current F1 = 0.6885245901639344  Max F1:  0.7575757575757577  Current F2 = 0.6213017751479291  Max F2:  0.7183908045977012  AUC at train max = 0.7777777777777778
Epoch 18/100
2942/2942 [==============================] - 7s 2ms/step - loss: 0.0381 - acc: 0.9901 - val_loss: 0.2740 - val_acc: 0.9471
TRAIN: Current SENS = 0.9955527318932655  Max SENS = 1.0  Current AUC = 0.9937558981103755  Max AUC = 0.9996345029239766  Current F1 = 0.9942893401015228  Max F1 = 0.9996824388694823  Current F2 = 0.9950469900939802  Max F2 = 0.9998729513403632  Current Loss = 0.03810792890415963  Min Loss = 0.004472043859075317
TEST: Current SENS = 0.6388888888888888  Max SENS = 0.6944444444444444  Current AUC = 0.8092105263157895  Max AUC:  0.8399122807017544  Current F1 = 0.696969696969697  Max F1:  0.7575757575757577  Current F2 = 0.660919540229885  Max F2:  0.718390804597

2942/2942 [==============================] - 7s 2ms/step - loss: 7.1333e-04 - acc: 1.0000 - val_loss: 0.3057 - val_acc: 0.9497
Update new train model: AUC = 1.0 --> 1.0
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  Current Loss = 0.0007133330745963768  Min Loss = 0.0007133330745963768
TEST: Current SENS = 0.5555555555555556  Max SENS = 0.6944444444444444  Current AUC = 0.7733918128654971  Max AUC:  0.8399122807017544  Current F1 = 0.6779661016949152  Max F1:  0.7575757575757577  Current F2 = 0.5988023952095808  Max F2:  0.7183908045977012  AUC at train max = 0.7733918128654971
Epoch 30/100
2942/2942 [==============================] - 7s 2ms/step - loss: 0.0124 - acc: 0.9969 - val_loss: 0.3485 - val_acc: 0.9524
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  Current Loss = 0.01235980949004356  Min Loss

2942/2942 [==============================] - 7s 2ms/step - loss: 5.9996e-04 - acc: 1.0000 - val_loss: 0.2683 - val_acc: 0.9524
Update new train model: AUC = 1.0 --> 1.0
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  Current Loss = 0.0005999555012206772  Min Loss = 0.0005999555012206772
TEST: Current SENS = 0.5833333333333334  Max SENS = 0.6944444444444444  Current AUC = 0.787280701754386  Max AUC:  0.8399122807017544  Current F1 = 0.7000000000000001  Max F1:  0.7575757575757577  Current F2 = 0.6250000000000001  Max F2:  0.7183908045977012  AUC at train max = 0.787280701754386
Epoch 42/100
2942/2942 [==============================] - 7s 2ms/step - loss: 4.5660e-04 - acc: 1.0000 - val_loss: 0.2928 - val_acc: 0.9497
Update new train model: AUC = 1.0 --> 1.0
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  

...

2942/2942 [==============================] - 7s 2ms/step - loss: 0.0974 - acc: 0.9711 - val_loss: 0.0424 - val_acc: 0.9974
Update new train model: AUC = 0.9698318620566665 --> 0.9731250510859955
TRAIN: Current SENS = 0.9491740787801779  Max SENS = 0.9491740787801779  Current AUC = 0.9731250510859955  Max AUC = 0.9731250510859955  Current F1 = 0.97265625  Max F1 = 0.97265625  Current F2 = 0.9584295612009237  Max F2 = 0.9584295612009237  Current Loss = 0.09740531013520051  Min Loss = 0.09740531013520051
TEST: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 0.9985380116959065  Max AUC:  0.9985380116959065  Current F1 = 0.9863013698630138  Max F1:  0.9863013698630138  Current F2 = 0.994475138121547  Max F2:  0.994475138121547  AUC at train max = 0.9985380116959065
Epoch 7/100
2942/2942 [==============================] - 7s 2ms/step - loss: 0.0928 - acc: 0.9742 - val_loss: 0.1630 - val_acc: 0.9709
Update new train model: AUC = 0.9731250510859955 --> 0.97213398277566
TRAIN: Current SENS = 

TEST: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 0.9970760233918129  Max AUC:  0.9985380116959065  Current F1 = 0.972972972972973  Max F1:  0.9863013698630138  Current F2 = 0.989010989010989  Max F2:  0.994475138121547  AUC at train max = 0.9970760233918129
Epoch 17/100
2942/2942 [==============================] - 7s 2ms/step - loss: 0.0544 - acc: 0.9816 - val_loss: 0.0368 - val_acc: 0.9947
Update new train model: AUC = 0.9809709311397936 --> 0.9877853385050938
TRAIN: Current SENS = 0.9777636594663278  Max SENS = 0.9777636594663278  Current AUC = 0.9877853385050938  Max AUC = 0.9877853385050938  Current F1 = 0.9878048780487806  Max F1 = 0.9878048780487806  Current F2 = 0.9817555498851748  Max F2 = 0.9817555498851748  Current Loss = 0.054359577479060006  Min Loss = 0.054359577479060006
TEST: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 0.9970760233918129  Max AUC:  0.9985380116959065  Current F1 = 0.972972972972973  Max F1:  0.9863013698630138  Current F2 = 0.989010989010989

2942/2942 [==============================] - 7s 2ms/step - loss: 0.0084 - acc: 0.9980 - val_loss: 0.0016 - val_acc: 1.0000
Update new train model: AUC = 0.9930114358322745 --> 0.9996345029239766
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 0.9996345029239766  Max AUC = 0.9996345029239766  Current F1 = 0.9996824388694823  Max F1 = 0.9996824388694823  Current F2 = 0.9998729513403632  Max F2 = 0.9998729513403632  Current Loss = 0.00841775126954671  Min Loss = 0.00841775126954671
TEST: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC:  1.0  Current F1 = 1.0  Max F1:  1.0  Current F2 = 1.0  Max F2:  1.0  AUC at train max = 1.0
Epoch 29/100
2942/2942 [==============================] - 7s 2ms/step - loss: 0.0032 - acc: 0.9997 - val_loss: 0.0077 - val_acc: 0.9974
Update new train model: AUC = 0.9996345029239766 --> 0.9996345029239766
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 0.9996345029239766  Max AUC = 0.9996345029239766  Current F1 = 0.99968243886948

2942/2942 [==============================] - 7s 2ms/step - loss: 0.0027 - acc: 0.9993 - val_loss: 0.0011 - val_acc: 1.0000
Update new train model: AUC = 0.9992690058479532 --> 1.0
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  Current Loss = 0.0026900969980060445  Min Loss = 0.0026900969980060445
TEST: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC:  1.0  Current F1 = 1.0  Max F1:  1.0  Current F2 = 1.0  Max F2:  1.0  AUC at train max = 1.0
Epoch 41/100
2942/2942 [==============================] - 7s 2ms/step - loss: 0.0040 - acc: 0.9993 - val_loss: 0.0011 - val_acc: 1.0000
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  Current Loss = 0.004040384028685602  Min Loss = 0.0026900969980060445
TEST: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC:  1.0  Current F1 = 1.0  Max 

2942/2942 [==============================] - 7s 2ms/step - loss: 2.8453e-04 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 0.9974
Update new train model: AUC = 1.0 --> 1.0
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  Current Loss = 0.0002845326681862909  Min Loss = 0.0002845326681862909
TEST: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 0.9985380116959065  Max AUC:  1.0  Current F1 = 0.9863013698630138  Max F1:  1.0  Current F2 = 0.994475138121547  Max F2:  1.0  AUC at train max = 0.9985380116959065
Epoch 55/100
2942/2942 [==============================] - 7s 2ms/step - loss: 2.6998e-04 - acc: 1.0000 - val_loss: 0.0040 - val_acc: 0.9974
Update new train model: AUC = 1.0 --> 1.0
TRAIN: Current SENS = 1.0  Max SENS = 1.0  Current AUC = 1.0  Max AUC = 1.0  Current F1 = 1.0  Max F1 = 1.0  Current F2 = 1.0  Max F2 = 1.0  Current Loss = 0.00026998157117422893  Min Loss = 0.00026998157117422893
T

# Evaluate full model with each subdataset 

In [32]:
def xt_eval_subset(X_test, y_test, best_model_name, bs):        
    # Load best model
    best_model = load_model(best_model_name)

    # Loss, Accuracy
    # Ref: https://keras.io/models/model/
    test_loss, test_acc = best_model.evaluate(X_test, 
                                              y_test,
                                              batch_size=bs)

    # Ref: https://keras.io/models/model/
    y_pred = best_model.predict(X_test, 
                                batch_size=bs, 
                                verbose=0, 
                                steps=None)

    # Flatten y_test and y_pred
    #print('------ Debug: y_test.shape =', y_test.shape)
    dim = y_test.shape 
    y_pred_ = y_pred.reshape(dim[0] * dim[1])
    y_pred_ = np.round(y_pred_)
    y_test_ = y_test.reshape(dim[0] * dim[1])


    # TN, FP, FN, TP, AUC, sensitivity, specificity, AUC
    # Ref: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    tn, fp, fn, tp = confusion_matrix(y_test_, y_pred_).ravel()
    sens = tp/(tp + fn)
    spec = tn/(fp + tn)
    auc = roc_auc_score(y_test_, y_pred_)
    
    print('test_loss, test_acc, tp, fn, tn, fp, sens, spec, auc =', test_loss, test_acc, tp, fn, tn, fp, sens, spec, auc)
    
    return test_loss, test_acc, tp, fn, tn, fp, sens, spec, auc 
       
# ==================================================
# Test function
file_in = 'D:\\Romanlab\XT_DataSet\dataset2\Dog_1OA\\test_2'

bs = 5
n_timestep = 400

#
X_test = np.loadtxt(file_in, delimiter=' ', usecols=(range(0, 16)))
y_test = np.loadtxt(file_in, delimiter=' ', usecols=(16))

#
dim = X_test.shape
print(dim)
X_test = X_test.reshape((int(dim[0]/n_timestep), n_timestep, 16))
y_test = y_test.reshape((int(dim[0]/n_timestep), n_timestep, 1))

#
xt_eval_subset(X_test, y_test, './Model/best-model--date--id.k.bs.cw.hu1.hu2.hu3--2018.07.13.03.20--Dog_.2.8.1.32.32.32', bs) 
#xt_eval_subset(X_test, y_test, './Model/best-model--date--id.k.bs.cw.hu1.hu2.hu3--2018.07.12.12.05--Dog_.1.8.1.8.0.0', bs) 
#xt_eval_subset(X_test, y_test, './Model/best-model--date--id.k.bs.cw.hu1.hu2.hu3--2018.07.12.12.45--Dog_.2.8.1.8.0.0', bs) 
#xt_eval_subset(X_test, y_test, './Model/best-model--date--id.k.bs.cw.hu1.hu2.hu3--2018.07.12.13.20--Dog_.3.8.1.8.0.0', bs) 
#xt_eval_subset(X_test, y_test, './Model/best-model--date--id.k.bs.cw.hu1.hu2.hu3--2018.07.12.13.56--Dog_.4.8.1.8.0.0', bs) 
print('Done')

(48000, 16)
120/120 [==============================] - 4s 30ms/step
test_loss, test_acc, tp, fn, tn, fp, sens, spec, auc = 0.5099687992284695 0.8484374994392662 7164 7236 33561 39 0.4975 0.9988392857142857 0.7481696428571429
120/120 [==============================] - 4s 30ms/step
test_loss, test_acc, tp, fn, tn, fp, sens, spec, auc = 0.2320444822932283 0.9113958328962326 10248 4152 33499 101 0.7116666666666667 0.9969940476190476 0.8543303571428572
120/120 [==============================] - 4s 31ms/step
test_loss, test_acc, tp, fn, tn, fp, sens, spec, auc = 0.37170639090860885 0.8699791658048829 8383 6017 33376 224 0.5821527777777777 0.9933333333333333 0.7877430555555555
120/120 [==============================] - 4s 31ms/step
test_loss, test_acc, tp, fn, tn, fp, sens, spec, auc = 0.4781038826331496 0.8673750019321839 8073 6327 33561 39 0.560625 0.9988392857142857 0.7797321428571428
120/120 [==============================] - 4s 31ms/step
test_loss, test_acc, tp, fn, tn, fp, sens, spec, a

In [25]:
not True

False